# Combinar conjuntos de datos: Concat (anteriormente conocido como append)

Algunos de los estudios de datos más interesantes proceden de la combinación de distintas fuentes de datos.
Estas operaciones pueden implicar cualquier cosa, desde una concatenación muy directa de dos conjuntos de datos diferentes, hasta uniones y fusiones más complicadas al estilo de las bases de datos que manejan correctamente cualquier solapamiento entre los conjuntos de datos.
``Series`` y ``DataFrame`` se construyen con este tipo de operaciones en mente, y Pandas incluye funciones y métodos que hacen que este tipo de manipulación de datos sea rápida y sencilla.

Aquí echaremos un vistazo a la simple concatenación de ``Series`` y ``DataFrame``s con la función ``pd.concat``; más tarde nos emplearemos en fusiones y uniones en memoria más sofisticadas implementadas en Pandas.

Comenzamos con las importaciones estándar:

In [1]:
import pandas as pd
import numpy as np

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_20672\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Por conveniencia, definiremos esta función que crea un ``DataFrame`` de una forma particular que será útil más adelante:

In [2]:
def make_df(cols, ind):
    """Crea rápidamente un DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# ejemplo DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Además, crearemos una clase rápida que nos permita mostrar múltiples ``DataFrame`` uno al lado del otro. El código hace uso del método especial ``_repr_html_``, que IPython utiliza para implementar su visualización de objetos enriquecidos:

In [3]:
class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

Su utilidad quedará más clara a medida que avancemos en el siguiente apartado.

## Recall: Concatenación de matrices NumPy

La concatenación de objetos ``Series`` y ``DataFrame`` es muy similar a la concatenación de arrays Numpy, que se puede hacer a través de la función ``np.concatenate`` como se discutió en el notebook dedicado a Numpy.

Recuerda que con ella puedes combinar el contenido de dos o más arrays en un único array:

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista o tupla de matrices a concatenar.
Además, toma una palabra clave ``axis`` que permite especificar el eje a lo largo del cual se concatenará el resultado:

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

In [6]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
xy = [x,y]
print(xy)
np.vstack([xy, z])

[[1, 2, 3], [4, 5, 6]]


array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

## Concatenación simple con ``pd.concat``

Pandas tiene una función, ``pd.concat()``, que tiene una sintaxis similar a ``np.concatenate`` pero contiene una serie de opciones que discutiremos momentáneamente:

``pd.concat()`` puede utilizarse para una simple concatenación de objetos ``Series`` o ``DataFrame``, al igual que ``np.concatenate()`` puede utilizarse para simples concatenaciones de arrays:

In [7]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
print(ser1)
ser2 = pd.Series(['D', 'E', 'F'], index=[1, 2, 3])
print(ser2)
pd.concat([ser1, ser2], axis = 0)

1    A
2    B
3    C
dtype: object
1    D
2    E
3    F
dtype: object


1    A
2    B
3    C
1    D
2    E
3    F
dtype: object

In [8]:
pd.concat([ser1, ser2], axis = 0).reset_index(drop=True)

0    A
1    B
2    C
3    D
4    E
5    F
dtype: object

También funciona para concatenar objetos de mayor dimensión, como ``DataFrame``s:

In [11]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [1, 2])

display('df1','df2','pd.concat([df1, df2], axis=1)')

df1
    A   B
1  A1  B1
2  A2  B2

df2
    A   B
1  A1  B1
2  A2  B2

pd.concat([df1, df2], axis=1)
    A   B   A   B
1  A1  B1  A1  B1
2  A2  B2  A2  B2

In [13]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [1, 2])

display('df1','df2','pd.concat([df1, df2], axis="rows")')

,A,B
1,A1,B1
2,A2,B2
,A,B
1,A1,B1
2,A2,B2
,A,B
1,A1,B1
2,A2,B2
1,A1,B1
2,A2,B2


Por defecto, la concatenación se realiza por filas dentro del ``DataFrame`` (es decir, ``axis=0``).
Al igual que ``np.concatenate``, ``pd.concat`` permite especificar un eje a lo largo del cual se producirá la concatenación.

Considere el siguiente ejemplo:

In [21]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=0)")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis=0)
     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1

In [22]:
pd.merge(df3, df4, left_index=True, right_index=True)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


In [24]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

Podríamos haber especificado ``axis=1``; aquí hemos utilizado el más intuitivo ``axis='col'``.

In [27]:
# Ejemplo con datos del INE
postal_1 = pd.read_excel("data/codigopostal_0_9.xlsx")
postal_2 = pd.read_excel("data/codigopostal_10_19.xlsx") # ¿Cuánto ocupa este fichero?

In [28]:
postal_1.head()

,CODAUTO,CPRO,CMUN,DC,NOMBRE
0,8,2,1,9,Abengibre
1,8,2,2,4,Alatoz
2,8,2,3,0,Albacete
3,8,2,4,5,Albatana
4,8,2,5,8,Alborea


In [29]:
postal_2.head()

,CODAUTO,CPRO,CMUN,DC,NOMBRE
0,16,1,51,3,Agurain/Salvatierra
1,16,1,1,4,Alegría-Dulantzi
2,16,1,2,9,Amurrio
3,16,1,49,3,Añana
4,16,1,3,5,Aramaio


In [30]:
postal = pd.concat([postal_1, postal_2])

In [ ]:
postal.head()

In [ ]:
postal.tail()

### Duplicate indices

Una diferencia importante entre ``np.concatenate`` y ``pd.concat`` es que la concatenación en Pandas *preserva los índices*, ¡incluso si el resultado tiene índices duplicados!
Considera este sencillo ejemplo:

In [31]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # ¡haz índices duplicados!
display('x', 'y', 'pd.concat([x, y], axis = 0)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


In [34]:

postal_ = np.concatenate([x, y])

In [35]:
postal_

array([['A0', 'B0'],
       ['A1', 'B1'],
       ['A2', 'B2'],
       ['A3', 'B3']], dtype=object)

Observe los índices repetidos en el resultado.
Aunque esto es válido dentro de ``DataFrame``s, el resultado es a menudo indeseable.
``pd.concat()`` nos da algunas maneras de manejarlo.

#### Captura de las repeticiones como error

Si quieres simplemente verificar que los índices en el resultado de ``pd.concat()`` no se solapan, puedes especificar la bandera ``verify_integrity``.
Con True, la concatenación lanzará una excepción si hay índices duplicados.
He aquí un ejemplo, en el que para mayor claridad capturaremos e imprimiremos el mensaje de error:

In [ ]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

Estoy solo tiene sentido cuando los índices son relevantes en la estructura de datos.
En este ejemplo de códigos postales no tiene sentido, ya que el índice solo refleja el orden de las filas y no aporta valor 

In [36]:
pd.concat([postal_1, postal_2], verify_integrity=True)

ValueError: Indexes have overlapping values: Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2156, 2157, 2158, 2159, 2160, 2161, 2162, 2163, 2164, 2165],
      dtype='int64', length=2166)

#### Ignorar el índice

A veces, el índice en sí no importa y se prefiere ignorarlo.
Esta opción puede especificarse utilizando el indicador ``ignorar_índice``.
Si se establece en true, la concatenación creará un nuevo índice entero para la ``Serie`` resultante:

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

In [37]:
pd.concat([postal_1, postal_2], ignore_index=True)

,CODAUTO,CPRO,CMUN,DC,NOMBRE
0,8,2,1,9,Abengibre
1,8,2,2,4,Alatoz
2,8,2,3,0,Albacete
3,8,2,4,5,Albatana
4,8,2,5,8,Alborea
...,...,...,...,...,...
8126,16,48,25,5,Zeberio
8127,16,48,913,0,Zierbena
8128,16,48,915,8,Ziortza-Bolibar
8129,18,51,1,3,Ceuta


In [38]:
pd.concat([df3, df4], ignore_index=True)

,A,B,C,D
0,A0,B0,NaN,NaN
1,A1,B1,NaN,NaN
2,NaN,NaN,C0,D0
3,NaN,NaN,C1,D1


In [39]:
df3

,A,B
0,A0,B0
1,A1,B1


In [40]:
df4

,C,D
0,C0,D0
1,C1,D1


#### Añadir claves MultiIndex

Otra opción es utilizar la opción ``keys`` para especificar una etiqueta para las fuentes de datos; el resultado será una serie indexada jerárquicamente que contendrá los datos:

In [ ]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

In [ ]:
# Puede ser útil si quieres preservar el fichero/objeto pandas del que provienen los datos
pd.concat([postal_1, postal_2],  keys=['postal_1', 'postal_2'])

El resultado es un ``DataFrame`` con múltiples índices, y podemos utilizar las herramientas comentadas en [Hierarchical Indexing](05_Hierarchical-Indexing.ipynb) para transformar estos datos en la representación que nos interesa.

### Concatenación con uniones

En los ejemplos sencillos que acabamos de ver, concatenábamos principalmente ``DataFrame``s con nombres de columna compartidos.
En la práctica, los datos de diferentes fuentes pueden tener diferentes conjuntos de nombres de columna, y ``pd.concat`` ofrece varias opciones en este caso.
Considera la concatenación de los siguientes dos ``DataFrame``s, que tienen algunas (¡pero no todas!) columnas en común:

In [41]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
# display(df5)
# display(df6)
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

Por defecto, las entradas para las que no hay datos disponibles se rellenan con valores NA.
Para cambiar esto, podemos especificar una de varias opciones para los parámetros ``join`` y ``join_axes`` de la función concatenar.
Por defecto, la unión es una unión de las columnas de entrada (``join='outer'``), pero podemos cambiar esto a una intersección de las columnas usando ``join='inner'``:

In [42]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='outer')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='outer')
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

In [43]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

In [ ]:
pd.concat([postal_1, postal_2] )

In [ ]:
postal_1['ID_MUNICIPIO'] = postal_1['CODAUTO'].astype(str).str.zfill(2)+ postal_1['CMUN'].astype(str).str.zfill(3)

In [ ]:
postal_1.head()

In [ ]:
pd.concat([postal_1, postal_2], join='inner')

La combinación de opciones de la función ``pd.concat`` permite una amplia gama de comportamientos posibles al unir dos conjuntos de datos; téngalos en cuenta cuando utilice estas herramientas para sus propios datos.